# Emissão de Notas Fiscais

Este notebook apresenta um processo de automação de emissão de notas fiscais usando o Python e o Selenium. As tarefas automatizadas são as seguintes:

- Acessar a página `login.html`

- Preenchimento dos campos do formulário de login

- Enviar o formulário de login

- Preencher os dados da nota fiscal na página `index.html`

- Enviar o formulário com os dados da nota fiscal (emitir nota fiscal)

## ⚠ Aviso

Todos os dados usados neste notebook são fictícios. Eles foram gerados no site [4Devs - Ferramentas Online](https://www.4devs.com.br/).

## Importações

In [1]:
# Importa o módulo os
import os

# Importa a biblioteca time
import time

# Importa o webdriver
from selenium import webdriver

# Importa a classe Service para especificar o caminho do webdriver
from selenium.webdriver.chrome.service import Service

# Importa o Chrome Driver Manager que nos permite baixar o webdriver compatível com a versão do navegador
from webdriver_manager.chrome import ChromeDriverManager

# Importa a classe By para os métodos de localização de elementos
from selenium.webdriver.common.by import By

# Importa a classe Keys que nos permite usar teclas do teclado
from selenium.webdriver.common.keys import Keys

# Importa a classe Select que nos permite trabalhar com menus drop-down
from selenium.webdriver.support.select import Select

## Configurações

### Criação de Pasta para Downloads

In [2]:
# Cria uma pasta com o nome especificado caso ela já não exista
if not os.path.exists('Notas Fiscais'):
    os.mkdir('Notas Fiscais')

### Opções de Download do Google Chrome

In [3]:
# Opções do Google Chrome
options = webdriver.ChromeOptions()

# Pasta para download das notas fiscais
pasta_notas_fiscais = os.path.join(os.getcwd(), 'Notas Fiscais')

# Configura as opções de download do Google Chrome
options.add_experimental_option('prefs', {
    'download.default_directory': pasta_notas_fiscais,
    'download.prompt_for_download': False,
    'download.directory_upgrade': True,
    'safebrowsing.enabled': True
    
})

### Opções de Download do Web Driver

In [4]:
# Instala o Chrome Driver e fornece o caminho do executável
servico = Service(ChromeDriverManager().install())

## Criação do Navegador

In [5]:
# Cria um navegador Google Chrome
navegador = webdriver.Chrome(service=servico, options=options)

## Acessando Página de Login

In [6]:
# Caminho deste notebook
caminho_nb = os.getcwd()

# Nome do arquivo (página) a ser aberto
pagina_login = 'login.html'

# Caminho absoluto do arquivo
caminho_pagina_login = os.path.join(caminho_nb, pagina_login)

# Abre o arquivo no navegador
navegador.get(caminho_pagina_login)

## Preenchendo Formulário de Login

In [7]:
# Localiza o campo de login do formulário pelo XPATH
campo_login = navegador.find_element(By.XPATH, '/html/body/div/form/input[1]') 

# Preenche o campo de login (e-mail aleatório)
campo_login.send_keys('raimundo.osvaldo.dias@glaudeimar.com.br')

# Localiza o campo de senha do formulário pelo XPATH
campo_senha = navegador.find_element(By.XPATH, '/html/body/div/form/input[2]')

# Preenche o campo de senha (senha aleatória)
campo_senha.send_keys('IYhEUjOg8u')

# Localiza o botão de enviar o formulário
botao_enviar = navegador.find_element(By.XPATH, '/html/body/div/form/button')

# Clica no botão de enviar o formulário
botao_enviar.click()

## Importação da Base de Clientes

In [8]:
# Importa o pandas com o apelido pd
import pandas as pd

# Importa a base dados dos clientes
df_clientes = pd.read_excel('NotasEmitir.xlsx')

# Exibe as cinco primeiras linhas do dataframe
df_clientes.head()

,Cliente,CPF/CNPJ,CEP,Endereço,Bairro,Municipio,UF,Inscricao Estadual,Descrição,Quantidade,Valor Unitario,Valor Total
0,Valentina Márcia Melissa da Cunha,25379574076,73402036,Módulo Módulo 12,Estância Mestre D'Armas V (Planaltina),Brasília,DF,213873,Nota referente aos serviços prestados/produtos...,3,273,819
1,Lara Benedita Josefa Drumond,30227643160,78730170,Avenida Manoel de Araújo Piau,Jardim Belo Panorama,Rondonópolis,MT,194139,Nota referente aos serviços prestados/produtos...,6,730,4380
2,Silvana Melissa da Paz,36636402806,69308220,Rua Madre Rosa,Treze de Setembro,Boa Vista,RR,203703,Nota referente aos serviços prestados/produtos...,2,389,778
3,Pietro Diego César Monteiro,51701896486,60182541,Rua Sousa Bandeira,Antônio Diogo,Fortaleza,CE,326342,Nota referente aos serviços prestados/produtos...,2,652,1304
4,Tomás Gabriel Alves,59992546980,96830690,Rua Capitão Francisco Cândido de Castro Menezes,Aliança,Santa Cruz do Sul,RS,549783,Nota referente aos serviços prestados/produtos...,9,533,4797


## Preenchendo Dados da Nota Fiscal

O laço de repetição a seguir percorre cada uma das linhas do dataframe __df_clientes__ a fim de obter os dados de cada célula da linha e usá-los para preencher o formulário da nota fiscal. Repare que a cada iteração é necessário limpar os campos do formulário antes de realizar seu preenchimento para que os novos dados não sejam concatenados aos anteriormente preenchidos.

Além disso, note que foi necessário converter os dados dos campos de CEP, CPF/CNPJ, Inscrição Estadual, Quantidade, Valor Unitário e Total para string.

In [9]:
# Percorre as linhas do dataframe
for linha in df_clientes.index:
    # Campo Nome
    navegador.find_element(By.NAME, 'nome').send_keys(df_clientes.loc[linha, 'Cliente'])

    # Campo Endereço
    navegador.find_element(By.NAME, 'endereco').send_keys(df_clientes.loc[linha, 'Endereço'])

    # Campo Bairro
    navegador.find_element(By.NAME, 'bairro').send_keys(df_clientes.loc[linha, 'Bairro'])

    # Campo Município
    navegador.find_element(By.NAME, 'municipio').send_keys(df_clientes.loc[linha, 'Municipio'])
    
    # Campo CEP
    navegador.find_element(By.NAME, 'cep').send_keys(str(df_clientes.loc[linha, 'CEP']))
    
    # Campo UF
    navegador.find_element(By.NAME, 'uf').send_keys(df_clientes.loc[linha, 'UF'])

    # Campo CPF/CNPJ
    navegador.find_element(By.NAME, 'cnpj').send_keys(str(df_clientes.loc[linha, 'CPF/CNPJ']))

    # Campo Inscrição Estadual
    navegador.find_element(By.NAME, 'inscricao').send_keys(str(df_clientes.loc[linha, 'Inscricao Estadual']))

    # Campo Descrição
    navegador.find_element(By.NAME, 'descricao').send_keys(df_clientes.loc[linha, 'Descrição'])

    # Campo Quantidade
    navegador.find_element(By.NAME, 'quantidade').send_keys(str(df_clientes.loc[linha, 'Quantidade']))

    # Campo Valor Unitário
    navegador.find_element(By.NAME, 'valor_unitario').send_keys(str(df_clientes.loc[linha, 'Valor Unitario']))

    # Campo Total
    navegador.find_element(By.NAME, 'total').send_keys(str(df_clientes.loc[linha, 'Valor Total']))
    
    # Botão Emitir nota
    navegador.find_element(By.CLASS_NAME, 'registerbtn').click()
    
    # Recarrega o navegador para limpar campos do formulário
    navegador.refresh()